In [1]:
#| default_exp feature.cards

In [2]:
#| exporti

from dataclasses import dataclass, field

from typing import List, Any
import datetime as dt
from numbers import Number

import mbison.client.core as dmda
import mbison.client.utils as dmut

from nbdev.showdoc import patch_to

In [3]:
#| hide
import nbdev
import os
from pprint import pprint

In [4]:
auth = dmda.DomoAuth(
    domo_instance=os.environ["DOMO_INSTANCE"],
    access_token=os.environ["DOMO_ACCESS_TOKEN"],
)
auth

DomoAuth(domo_instance='domo-community', username=None)

## Routes

In [5]:
card_id = '577316875'

In [6]:
#| exports
class Cards_API_Exception(dmda.API_Exception):
    def __init__(self, res, message=None):

        super().__init__(res=res, message=message)

In [7]:
# | exports

def generate_search_cards_only_apps_filter():
    return {
        "includeCardTypeClause": True,
        "cardTypes": ["domoapp", "mason", "custom"],
        "ascending": True,
        "orderBy": "cardTitle",
    }

def search_cards(
    auth: dmda.DomoAuth,
    query: dict = None,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_loop: bool = False,
    limit: int = 100,
    offset: int = 0,
):

    endpoint = "/api/content/v2/cards/adminsummary"
    query = query or {}
    


    def arr_fn(res):
        if res.status == 429:
            return []
        
        return res.response.get("cardAdminSummaries", [])
    
    res = dmda.looper(
        auth=auth,
        arr_fn=arr_fn,
        offset_params={"limit": "limit", "offset": "skip"},
        offset_params_is_header=True,

        request_type="POST",
        endpoint=endpoint,

        debug_api=debug_api,
        return_raw=return_raw,
        debug_loop=debug_loop, 

        # params = params

        body=query,
        limit=limit,
        offset=offset,

    )

    if not res.is_success:
        print(res)
        raise Cards_API_Exception(res=res)

    return res

In [8]:

query = generate_search_cards_only_apps_filter()

res = search_cards(
    auth=auth,
    query=query,
    return_raw=False,
    debug_loop=False,
    debug_api=False,
    offset = 0)

cards =res.response

print(len(cards))
cards[0:1]

191


[{'owners': [{'id': '1728973208',
    'type': 'USER',
    'displayName': 'Peter Shull'}],
  'id': 528432108,
  'type': 'domoapp',
  'badgeUpdated': 1711677072000,
  'title': '100 People',
  'locked': False,
  'pageHierarchy': [{'pageId': 682992709, 'title': 'LLM Testing'}]}]

In [9]:
# | exports

optional_parts = [
    "certification",
    "datasources",
    "domoapp",
    "drillPath",
    "masonData",
    "metadata",
    "owners",
    "problems",
    "properties",
]


def get_card_by_id(card_id, auth: dmda.DomoAuth, optional_parts = 'certification,datasources,drillPath,owners,properties,domoapp', debug_api: bool = False, return_raw: bool = False):
    endpoint = "/api/content/v1/cards/"

    params = {"parts": optional_parts, "urns": card_id}

    res = dmda.domo_api_request(

        auth=auth,

        request_type="GET",
        endpoint=endpoint,

        debug_api=debug_api,
        params=params,

    )

    if not res.is_success:
        raise Cards_API_Exception(res=res)
    
    if return_raw:
        return res
    
    res.response = res.response[0]
    

    return res

In [10]:
card= (get_card_by_id(card_id=cards[0]['id'], auth = auth)).response
card

{'domoapp': {'id': 'aace1266-dc7f-42cb-8595-46069d91a703'},
 'drillPath': {},
 'owners': [{'id': '1728973208',
   'type': 'USER',
   'displayName': 'Peter Shull'}],
 'datasources': [{'dataSourceId': 'b995e5ba-b1cc-4dff-89f7-d8949c0d0aad',
   'dataSourceName': 'Example Sales Data',
   'displayType': 'webform',
   'dataType': 'webform',
   'providerType': 'webform',
   'isSampleData': False,
   'lastUpdated': 1711668314801,
   'adc': False,
   'phase': None,
   'state': 'SUCCESS'},
  {'dataSourceId': '2b0016a7-02bd-45b4-a9fe-b33fbf5b365e',
   'dataSourceName': '100people',
   'displayType': 'large-file-upload',
   'dataType': 'large-file-upload',
   'providerType': 'large-file-upload',
   'isSampleData': False,
   'lastUpdated': 1711674871363,
   'adc': False,
   'phase': None,
   'state': 'SUCCESS'}],
 'certification': {'state': 'NOT_CERTIFIED', 'adminCertified': False},
 'urn': '528432108',
 'id': 528432108,
 'type': 'domoapp',
 'created': 1711668314,
 'badgeUpdated': 1711677072000,
 '

### classes

In [11]:
#| exports

@dataclass
class DomoCard:
    id: str
    auth: dmda.DomoAuth = field(repr=False)
    title: str = None
    description: str = None
    type: str = None
    urn: str = None
    chart_type: str = None
    dataset_id: str = None

    datastore_id : str = None
    
    domo_collections: List[Any] = None
    domo_source_code : Any = None

    owners: List[any] = None

    def display_url(self) -> str:
        return f"https://{self.auth.domo_instance}.domo.com/kpis/details/{self.id}"
    

    @classmethod
    def _from_json(cls, obj : dict, auth: dmda.DomoAuth):

        card = cls(
            auth=auth,
            id=obj['id'],
            title=obj['title'],
            type= obj['type'],
            urn= obj['urn'],
            description= obj.get('description'),
            owners = obj.get('owners')
        )

        if obj.get('domoapp',{}).get('id'):
            card.datastore_id = obj['domoapp']['id']
    
        return card

    @classmethod
    def get_by_id(
        cls,
        card_id: str,
        auth: dmda.DomoAuth,
        debug_api: bool = False,
        return_raw: bool = False
    ):
        res = get_card_by_id(
            auth=auth,
            card_id=card_id, 
            debug_api=debug_api
        )

        if return_raw:
            return res


        return cls._from_json(res.response, auth)


In [12]:
DomoCard.get_by_id(
    card_id=355758291,
    auth = auth,
    return_raw = False
)

DomoCard(id=355758291, title='Phoenix Stacked Bar Chart Brick', description=None, type='domoapp', urn='355758291', chart_type=None, dataset_id=None, datastore_id='be4d8af8-b48f-4ae8-ab5f-f010dcfd865f', domo_collections=None, domo_source_code=None, owners=[{'id': '1334846498', 'type': 'USER', 'displayName': 'Jonathan Pilafas'}])

In [16]:
#| exports

@patch_to(DomoCard)
def get_collections(self, debug_api : bool = False, return_raw: bool = False):
    import mbison.feature.appdb as dmdb
    
    res  = dmdb.get_collections(datastore_id= self.datastore_id, auth = auth, debug_api= debug_api)

    if return_raw:
        return res

    self.domo_collections = [ dmdb.AppDbCollection.get_by_id(collection_id = obj['id'], auth = auth, debug_api = debug_api) for obj in res.response]

    return self.domo_collections


@patch_to(DomoCard)
def get_source_code(self, debug_api : bool = False):
    self.get_collections(debug_api = debug_api)

    collection_name = 'ddx_app_client_code'
    code_collection = next((domo_collection for domo_collection in self.domo_collections if domo_collection.name ==collection_name), None)

    if not code_collection:
        raise dmda.Class_Exception(cls = self, message = f"collection - {collection_name} not found")
    
    self.domo_source_code = code_collection.query_documents(debug_api = debug_api)[0]

    return self.domo_source_code


In [19]:
domo_card = DomoCard.get_by_id(
    card_id=577316875,
    auth = auth,
    return_raw = False
)

domo_source_code_document = domo_card.get_source_code()
pprint(domo_source_code_document.content)

{'htmlBlank': {'css': '',
               'html': '',
               'js': '// DDX Bricks Wiki - See '
                     'https://developer.domo.com/docs/ddx-bricks/getting-started-using-ddx-bricks\n'
                     '// for tips on getting started, linking to Domo data and '
                     'debugging your app\n'
                     ' \n'
                     '//Available globals\n'
                     'var domo = window.domo; // For more on domo.js: '
                     'https://developer.domo.com/docs/dev-studio-guides/domo-js#domo.get\n'
                     'var datasets = window.datasets;\n'
                     '\n'
                     '//Step 1. Select your dataset(s) from the button in the '
                     'bottom left corner\n'
                     '\n'
                     '\n'
                     '\n'
                     '//Step 2. Query your dataset(s): '
                     'https://developer.domo.com/docs/dev-studio-references/data-api\n'
      

In [15]:
#| hide

nbdev.nbdev_export('./cards.ipynb')